In [1]:
!pip install bs4

In [2]:
import urllib.request as urllib2
import csv
from bs4 import BeautifulSoup
import requests

In [3]:
# send a GET request to the webpage
url = "https://educationnewshub.co.ke/agriculture-revision-questions-and-answers-f1-4/"
response = requests.get(url)

# create a BeautifulSoup object to parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')

In [4]:
# Add a User-Agent header to identify yourself as a web browser
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36"
}

# Make a GET request to the website
response = requests.get("https://www.tractorjunction.com/blog/top-agriculture-questions-and-answers/", headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content of the website
    soup = BeautifulSoup(response.text, 'html.parser')



In [5]:
# Formating the parsed html file
strhtm = soup.prettify()

In [6]:
# Print few lines
print (strhtm[:1000000])

<!DOCTYPE html>
<html lang="en-US" prefix="og: //ogp.me/ns#">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="true" name="HandheldFriendly"/>
  <meta content="index, follow, max-snippet:-1, max-image-preview:large, max-video-preview:-1" name="robots">
   <link href="//gmpg.org/xfn/11" rel="profile"/>
   <link href="//fonts.googleapis.com/css?family=Roboto:400,700" rel="stylesheet"/>
   <link as="font" crossorigin="" href="//www.tractorjunction.com/blog/wp-content/themes/revenue/assets/fonts/mulish-semibold.woff2" rel="preload" type="font/woff2"/>
   <link href="//cdn.jsdelivr.net/npm/bootstrap@4.6.0/dist/css/bootstrap.min.css" rel="stylesheet"/>
   <link href="//www.tractorjunction.com/blog/wp-content/themes/revenue/assets/css/main.min.css?v=1.7.7" rel="stylesheet">
    <link crossorigin="anonymous" href="//stackpath.bootstrapcdn.com/font-awesome/4.7.0/css

In [7]:
questions = [question.text for question in soup.find_all('h4')]
print(questions)
answers = []

for b_tag in soup.find_all('b', text='Ans.'):
    answerset =b_tag.find_all_next('span')
    for answer in answerset:
       answer= answer.text.strip()
       answers.append(answer)

print(answers)

# write to CSV file
with open('qa.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Question', 'Answer'])
    for i in range(len(questions)):
        writer.writerow([questions[i], answers[i]])


['Q.1 Who invented the first modern tractor?', 'Q.2 Which tractor is best and most powerful?', 'Q.3 What is the use of PTO HP in Tractor?', 'Q.4 What is the difference between a Mini and Utility tractor?', 'Q.5 What is HP in a tractor?', 'Q.6 What is the torque in a tractor?', 'Q.7 What is the RPM of the tractor?', 'Q.8 Which is the top selling tractor in India?', 'Q.9 How many types of tractor are used in agriculture?', 'Q.10 What is the HP range in a mini tractor?', 'Q.11 What is cc in tractor engine?', 'Q.12 How do you maintain a tractor?', 'Q.13 How important is 4 Wheel Drive in tractors?', 'Q.14 Which tractor is best for 2 acres land?', 'Q.15\xa0What is the difference between displacement capacity and CC of a farm tractor?', 'Q.16\xa0How can I get a Used Tractor in India?', 'Q.17\xa0What to check before buying a used tractor?', 'Q.18\xa0Why is a three-point hitch important in a farm tractor?', 'Q.19 How many different types of farming methods are there in India?', 'Q.20 What are t

In [8]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments

In [9]:
def load_dataset(file_path, tokenizer, block_size = 128):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset

In [10]:
def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, 
        mlm=mlm,
    )
    return data_collator

In [4]:
def train(train_file_path,model_name,
          output_dir,
          overwrite_output_dir,
          per_device_train_batch_size,
          num_train_epochs,
          save_steps):
  tokenizer = GPT2Tokenizer.from_pretrained(model_name)
  train_dataset = load_dataset(train_file_path, tokenizer)
  data_collator = load_data_collator(tokenizer)

  tokenizer.save_pretrained(output_dir)
      
  model = GPT2LMHeadModel.from_pretrained(model_name)

  model.save_pretrained(output_dir)

  training_args = TrainingArguments(
          output_dir=output_dir,
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=per_device_train_batch_size,
          num_train_epochs=num_train_epochs,
      )

  trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
  )
      
  trainer.train()
  trainer.save_model()

train_file_path = "qa.csv"
model_name = 'gpt2'
output_dir = 'result'
overwrite_output_dir = False
per_device_train_batch_size = 8
num_train_epochs = 5.0
save_steps = 500

train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

C:\ProgramData\Anaconda3\lib\site-packages\transformers\data\datasets\language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 39
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 25
  Number of trainable parameters = 124439808


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to result
Configuration saved in result\config.json
Configuration saved in result\generation_config.json
Model weights saved in result\pytorch_model.bin


In [3]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, GPT2TokenizerFast, GPT2Tokenizer

def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model


def load_tokenizer(tokenizer_path):
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
    return tokenizer


def generate_text(sequence, max_length):
    model_path = "result"
    model = load_model(model_path)
    tokenizer = load_tokenizer(model_path)
    ids = tokenizer.encode(f'{sequence}', return_tensors='pt')
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        pad_token_id=model.config.eos_token_id,
        top_k=50,
        top_p=0.95,
    )
    print(tokenizer.decode(final_outputs[0], skip_special_tokens=True))

sequence = input()
max_len = int(input())
generate_text(sequence, max_len)

Q.85 Which food does India export the most?', 
100
Q.85 Which food does India export the most?',  Ivy-ripe strawberries cost about $10 and are available for about half an acre of fruit," said Arun. The price of strawberry production and distribution is high in India which also includes rural areas,"
"There are no reliable sources of water for strawberries, so farmers only use irrigated land for their crops," said Arun.
"It is best to produce locally grown strawberries at a lower cost," added Ar
